# modelforge.curate : Basic Usage

This notebook will demonstrate basic usage of the curate module in modelforge, developed to make it easier to create datasets with a uniform structure, compatible with modelforge. 

In the curate module, we have 3 levels of hierarchy: 

- dataset (i.e., an instance of `SourceDataset`);
- the dataset contains records (instances of the `Record` class)
- each record contains properties (instances of the various property pydantic model classes, which all inherit for the `RecordProperty` class)

The use of pydantic for properties aids in validation of the inputs. 

In [1]:
from modelforge.curate import Record, SourceDataset
from modelforge.curate.units import GlobalUnitSystem
from modelforge.curate.properties import AtomicNumbers, Positions, Energies, Forces, MetaData

from openff.units import unit

import numpy as np

### Set up a new dataset
To start, we will create a new instance of the `SourceDataset` class to store the dataset.

In [2]:
new_dataset = SourceDataset("test_dataset")

### Create a record

To create a record, we just instantiate the `Record` class providing a unique name as a string; this name will be used within the dataset to access/update records. 

In [3]:
record_mol1 = Record('mol1')

### Define properties
Each record must include a few basic elements to be considered complete, namely:
- atomic numbers
- positions
- energies
  
Records may of course contain other properties/metadata, but this is the minimal set of information used in modelforge during training. The curate packages provides pydantic models for these and other common properties that appear in datasets.  

#### Defining atomic numbers
Let us first start by considering how to initialize atomic numbers, in this case for an example CH molecule:

In [4]:
atomic_numbers = AtomicNumbers(value=np.array([[1], [6]]))

The array that is should have shape (n_atoms, 1).  An error will be raised if `len(value.shape) != 2` or `value.shape[1] != 1`. 

`AtomicNumbers` can accept either a numpy array or a python list as input (it will be converted to a numpy array internally). The following syntax will produce an equivalent instance:

In [5]:
atomic_numbers = AtomicNumbers(value=[[1], [6]])

#### Defining positions

To define positions, we will use the `Positions` pydantic model.  Since positions should have units associated with them, they must also be set at the time of initialization. 

Units can be passed as an openff.units `Unit` or a string that can be understood by openff.units. An error will be raised if units are not defined. 

Positions are a per_atom property and thus must be a 3d array with shape (n_configs, n_atoms, 3).
If `value.shape[2] !=3` or `len(value.shape) != 3`, this will raise an error.  



In [8]:
positions = Positions(
    value=np.array([[[1.0, 1.0, 1.0], [2.0, 2.0, 2.0]]]), 
    units="nanometer"
)

In [9]:
positions

Positions(name='positions', value=array([[[1., 1., 1.],
        [2., 2., 2.]]]), units=<Unit('nanometer')>, classification='per_atom', property_type='length', n_configs=1, n_atoms=2)

#### Defining energies 
To define energies, we will use the `Energies` pydantic model; as with positions, units must also be set.  

Note, energy is a per_system property and thus the shape of the input array must be (n_configs, 1); an error will be raised if `value.shape[1] !=1` or `len(value.shape) != 2`.

In [12]:
energies = Energies(
    value=np.array([[0.1]]), 
    units=unit.hartree
)

#### Other properties

Pydantic models have also been defined for other common properties:
- `Forces`
- `PartialCharges`
- `TotalCharge`
- `DipoleMoment`
- `QuadrupoleMoment`
- `Polarizability`
- `MetaData`

Note, each of thes emodels inherits from a more general `RecordProperty` pydantic model; this model can be used to define any additional properties, but requires the user to provide the classification (e.g., per_atom, per_system) and the type (for the purposes of unit conversion, e.g., length, energy, force, charge, etc.). This will be discussed separately.

### Add properties to a record

Having defined properties we can now add them to the record. Properties can be added individually to the record or provided as a list:

In [13]:
record_mol1.add_property(atomic_numbers)
record_mol1.add_properties([positions,energies])

By default when instantiating a new `Record` instance, `append_property = False`.
If `append_property == False`, an error will be raised if you try to add a property with the same name more than once to the same record. This ensures we do not accidentally overwrite data in a record.

They following will produce a ValueError because atomic numbers have already been set for the record

In [14]:
record_mol1.add_property(atomic_numbers)

ValueError: Atomic numbers already set for record mol1

## Add a record to a dataset

To add a record to the dataset, we use the `add_record` function of `SourceDataset`.

In [15]:
new_dataset.add_record(record_mol1)

### Viewing a record
Printing a record provides a summary of the contents.

In [16]:
print(record_mol1)

name: mol1
* n_atoms: 2
* n_configs: 1
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[1., 1., 1.],
        [2., 2., 2.]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=1 n_atoms=2
* per-system properties (['energies']):
 -  name='energies' value=array([[0.1]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
* meta_data: ([])



The record can also be exported to a dict. 

In [17]:
record_mol1.to_dict()

{'name': 'mol1',
 'n_atoms': 2,
 'n_configs': 1,
 'atomic_numbers': AtomicNumbers(name='atomic_numbers', value=array([[1],
        [6]]), units=<Unit('dimensionless')>, classification='atomic_numbers', property_type='atomic_numbers', n_configs=None, n_atoms=2),
 'per_atom': {'positions': Positions(name='positions', value=array([[[1., 1., 1.],
          [2., 2., 2.]]]), units=<Unit('nanometer')>, classification='per_atom', property_type='length', n_configs=1, n_atoms=2)},
 'per_system': {'energies': Energies(name='energies', value=array([[0.1]]), units=<Unit('hartree')>, classification='per_system', property_type='energy', n_configs=1, n_atoms=None)},
 'meta_data': {}}

We can easily view an individual record within a dataset using the `get_record` function in the `SourceDataset` class.

In [18]:
temp_record = new_dataset.get_record(record_name="mol1")
print(temp_record)

name: mol1
* n_atoms: 2
* n_configs: 1
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[1., 1., 1.],
        [2., 2., 2.]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=1 n_atoms=2
* per-system properties (['energies']):
 -  name='energies' value=array([[0.1]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
* meta_data: ([])



Note, that `get_record` returns a copy.  If this copy is modified, the `update_record` function should be used to updated it within the dataset. 


In [19]:
temp_record.per_atom['positions'].value = np.array([[[2,2,2], [3,3,3]]])

new_dataset.update_record(temp_record)

temp_record_updated = new_dataset.get_record(record_name="mol1")
print(temp_record_updated)

name: mol1
* n_atoms: 2
* n_configs: 1
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[2, 2, 2],
        [3, 3, 3]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=1 n_atoms=2
* per-system properties (['energies']):
 -  name='energies' value=array([[0.1]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
* meta_data: ([])



Note, the update function performs the update by looking at the `name` field in the record.  For example, if the name of the record is changed from "mol1" to "mol5", and "mol5" already exists in the dataset, it will be overwritten. Alternatively, if "mol5" does not exist, and error will be raised.

In this case, since "mol5" does not exist, an error is raised.

In [15]:
temp_record_updated.name = 'mol5'

new_dataset.update_record(temp_record_updated)

2025-02-06 18:50:33.127 | WARNING  | modelforge.curate.curate:update_record:479 - Record with name mol5 does not exist in the dataset. Use the add_record function.


ValueError: Record with name mol5 does not exist in the dataset.

If the name were changed, thus creating a new record, we would need to use the `add_record` function. 

In [16]:
new_dataset.add_record(temp_record_updated)
print(new_dataset.records.keys())

dict_keys(['mol1', 'mol5'])


To remove a record, we can use the `remove_record` function. 

In [17]:
new_dataset.remove_record('mol5')
print(new_dataset.records.keys())

dict_keys(['mol1'])


## Adding properties directly to a dataset

Rather than creating an instance of the `Record` class and adding this to the dataset, we can use the `SourceDataset` class directly. The functions in `SourceDataset` effectively just provide wrappers to the functions that exist within the `Record` class. As such, both approaches are equivalent but one may be more convenient depending on the structure of the original dataset that is being curated. 

The following code performs the same functionality to create a record named "mol1".

In [18]:
new_dataset2 = SourceDataset('test_dataset.hdf5')

new_dataset2.create_record('mol1')

new_dataset2.add_property('mol1', atomic_numbers)
new_dataset2.add_properties('mol1', [positions,energies])

temp_record = new_dataset2.get_record('mol1')

print(temp_record)

name: mol1
* n_atoms: 2
* n_configs: 1
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[1., 1., 1.],
        [2., 2., 2.]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=1 n_atoms=2
* per-system properties (['energies']):
 -  name='energies' value=array([[0.1]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
* meta_data: ([])



### Validating a record
Within these readouts, we see `n_atoms` and `n_configs` reported.  `n_atoms` is calculated from the dimensions of the atomic numbers; validation is then performed for all per_atom properties to ensure that all properties in the record have the same number of atoms.  Similarly, we validate that all per_system and per_atom properties have the same value of `n_configs` (determined by the first index of the shape of their arrays).  If the values were inconsistent, a descriptive message reporting the shape of each of the arrays would be provided. 

This validation can be triggered manually:

In [20]:
record_mol1.validate()

True

More complete validation can be performed at the dataset level. This validation includes checking for each record that:

- number of atoms is consistent
- number of configurations is consistent
- validation of units (e.g., that the unit provided for Positions is a length),
- ensuring that at minimum, atomic numbers, positions, and energies have been defined in the dataset

This can be done for individual records or on the entire dataset:

In [20]:
new_dataset.validate_record("mol1")

True

In [21]:
new_dataset.validate_records()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4848.91it/s]
2025-02-06 18:50:38.855 | INFO     | modelforge.curate.curate:validate_records:739 - All records validated successfully.


True

### Saving to an HDF5 file

To save ths to an hdf5 file, we call the `to_hdf5` function of the `SourceDataset` class, passing the output path and filename. This will automatically perform the validation discussed above before we write to the file. 

Additionally, when writing the file, it will convert records to a consistent unit system (by default, kilojoules_per_mole and nanometers are the base unit system for energy and distance), as defined by the `GlobalUnitSystem` class (discussed below).

In [22]:
new_dataset.to_hdf5(file_path="./", file_name="test_dataset.hdf5")

2025-02-06 18:50:39.877 | INFO     | modelforge.curate.curate:to_hdf5:778 - Validating records
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4471.54it/s]
2025-02-06 18:50:39.878 | INFO     | modelforge.curate.curate:validate_records:739 - All records validated successfully.
2025-02-06 18:50:39.878 | INFO     | modelforge.curate.curate:to_hdf5:781 - Writing records to HDF5 file
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

## Defining multiple properties of the same type

In the examples above, we did not provide a name to any of the properties we defined, intead using the default name in the pydantic model.  In some cases, we may wish to change this name, e.g., to match the name used in the original dataset or to be able to define multiple different energies (e.g., different contributions to the total energy, or calculated with different levels of theory). 

For example, let us consider that our dataset also includes a separate entry for the contribution of dispersion and we wish to store this with name 'energies_disp'.  We will add this to record "mol`" in the first dataset we defined.

Note, since this record already exists within the dataset, we can just use the `add_property` function to avoid having to call `get_record` and then `update_record` (as shown above). 

In [23]:
disp_energy = Energies(name='energies_disp', value=np.array([[0.03]]), units=unit.hartree)

new_dataset.add_property(
    record_name="mol1", 
    property=disp_energy
)

In [24]:
new_dataset.get_record('mol1')

name: mol1
* n_atoms: 2
* n_configs: 1
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[2, 2, 2],
        [3, 3, 3]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=1 n_atoms=2
* per-system properties (['energies', 'energies_disp']):
 -  name='energies' value=array([[0.1]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
 -  name='energies_disp' value=array([[0.03]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
* meta_data: ([])

Note validation to ensure that energies are included, only looks at the `property_type`, rather than looking for a specific string in the `name` that has been provided.  If an property with `property_type` that is 'energy', validation will pass. 

## Creating an appendable record/dataset

In some cases, we may not have data for all configurations saved within the same array (e.g., when fetching data from qcarchive). To aid in these dataset types, we can initialize our instance of `SourceDataset` with `append_property=True`.  In these cases, rather than providing an error if the same property is added twice to the dataset, we will instead append the data to the existing array. 

In [25]:
appendable_dataset = SourceDataset(dataset_name="appendable", append_property=True)

For simplicity, let us reuse the properties already defined above and add these to the new dataset.  Note if we do not call `create_record` first, it will automatically create the record inside the dataset, if it does not exist. 

In [26]:
appendable_dataset.add_properties("mol2", [energies, atomic_numbers, positions])

2025-02-06 18:50:44.072 | INFO     | modelforge.curate.curate:add_property:548 - Record with name mol2 does not exist in the dataset. Creating it now.


Let us now examine the contents of the record, where we see that we only have a single configuration. 

In [27]:
appendable_dataset.get_record('mol2')

name: mol2
* n_atoms: 2
* n_configs: 1
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[1., 1., 1.],
        [2., 2., 2.]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=1 n_atoms=2
* per-system properties (['energies']):
 -  name='energies' value=array([[0.1]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
* meta_data: ([])

If we add the energies and positions a second time, we should see that we now have 2 configurations.  

In [28]:
appendable_dataset.add_properties("mol2", [energies, positions])

In [29]:
appendable_dataset.get_record('mol2')

name: mol2
* n_atoms: 2
* n_configs: 2
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[1., 1., 1.],
        [2., 2., 2.]],

       [[1., 1., 1.],
        [2., 2., 2.]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=2 n_atoms=2
* per-system properties (['energies']):
 -  name='energies' value=array([[0.1],
       [0.1]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=2 n_atoms=None
* meta_data: ([])

Note, atomic_numbers cannot change within a record; as such, these do not have the concept of "n_configs" (shape is always [n_atoms,1]).  Atomic numbers therefore do not need to (and cannot) be appended to. 

The following attempts to append atomic numbers a second time, leading to an error being raised. 

In [30]:
appendable_dataset.add_property("mol2", atomic_numbers)

ValueError: Atomic numbers already set for record mol2

If we try to append a property that does not have the same number of atoms, an error will be raised. For example, below we try to append positions for a configuration with 3 atoms, not 2. 

In [30]:
positions2 = Positions(value=[[[1.0, 1.0, 1.0], [2.0, 2.0, 2.0], [3.0, 3.0, 3.0]]], units="angstrom")
appendable_dataset.add_property("mol2",  positions2)

AssertionError: mol2: n_atoms of positions does not: 3 != 2.

In our previous definition of energies in hatree and positions in nanometer; if we were to now define properties in different, yet compatible units, these values will be automatically converted to the existing units before appending.  

In [31]:
energies2 = Energies(value=np.array([[0.1]]), units=unit.kilojoules_per_mole)
positions2 = Positions(value=[[[1.0, 1.0, 1.0], [2.0, 2.0, 2.0]]], units="angstrom")
appendable_dataset.add_properties("mol2", [energies2, positions2])

If we now print the contents, we can see we now have 3 configurations, where the final values in the position array are a factor of 0.1 smaller as the base unit was nanometers and we defined above in angstrom; similarly the final energy in energies has been appropriate converted from kilojoules per mole to hartree to match the previously defined unit. 

In [32]:
appendable_dataset.get_record('mol2')

name: mol2
* n_atoms: 2
* n_configs: 3
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[1. , 1. , 1. ],
        [2. , 2. , 2. ]],

       [[1. , 1. , 1. ],
        [2. , 2. , 2. ]],

       [[0.1, 0.1, 0.1],
        [0.2, 0.2, 0.2]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=3 n_atoms=2
* per-system properties (['energies']):
 -  name='energies' value=array([[1.00000000e-01],
       [1.00000000e-01],
       [3.80879885e-05]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=3 n_atoms=None
* meta_data: ([])

An instance of `Record` can also be set to be appendable, allowing the same workflow as above, but operating on a record directly.  

In [33]:
new_record = Record('mol3', append_property=True)

new_record.add_properties([energies, atomic_numbers, positions])

new_record.add_properties([energies, positions])
new_record.add_properties([energies2, positions2])

print(new_record)


name: mol3
* n_atoms: 2
* n_configs: 3
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[1. , 1. , 1. ],
        [2. , 2. , 2. ]],

       [[1. , 1. , 1. ],
        [2. , 2. , 2. ]],

       [[0.1, 0.1, 0.1],
        [0.2, 0.2, 0.2]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=3 n_atoms=2
* per-system properties (['energies']):
 -  name='energies' value=array([[1.00000000e-01],
       [1.00000000e-01],
       [3.80879885e-05]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=3 n_atoms=None
* meta_data: ([])



## Unit system and unit validation

When defining individual properties, units are also validated.  When defining a property, users can specify any unit that is:
- (1) supported by openff.units
- (2) compatible with the parameter type (i.e., Positions expect a unit of length).

Bullet 2 is assessed by comparing to the default values in the `GlobalUnitSystem` class (note, we are not making any unit conversions at the point of initializing a record, just checking for compatibility). 

The following will fail validation because we expect positions to be defined in distance units. 

In [35]:
pos = Positions(value=[[[1.0, 1.0, 1.0], [2.0, 2.0, 2.0], [3.0, 3.0, 3.0]]], units=unit.angstrom*unit.angstrom)


ValidationError: 1 validation error for Positions
  Value error, Unit angstrom ** 2 of positions are not compatible with the property type length.
 [type=value_error, input_value={'value': [[[1.0, 1.0, 1....<Unit('angstrom ** 2')>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

Units are stored as class attributes within the `GlobalUnitSystem` class. 

In [36]:
from modelforge.curate.curate import GlobalUnitSystem
print(GlobalUnitSystem())

area : nanometer ** 2
atomic_numbers : dimensionless
charge : elementary_charge
dimensionless : dimensionless
dipole_moment : elementary_charge * nanometer
energy : kilojoule_per_mole
force : kilojoule_per_mole / nanometer
frequency : gigahertz
heat_capacity : kilojoule_per_mole / kelvin
length : nanometer
name : default
octupole_moment : elementary_charge * nanometer ** 3
polarizability : nanometer ** 3
quadrupole_moment : elementary_charge * nanometer ** 2
wavenumber : 1 / centimeter


Since these are class attributes, not instance variables, any changes or additions to the `GlobalUnitSystem `will apply to all usages within the script. For example, the following will change the units for length to angstroms. 

In [37]:
GlobalUnitSystem.set_global_units('length', unit.angstrom)

print(GlobalUnitSystem.get_units('length'))

angstrom


The `set_global_units` function can also be used to add in a new property_type and associated units.  For example, the following would add pressure as a possible property_type. 

In [38]:
GlobalUnitSystem.set_global_units('pressure', unit.atmosphere)

print(GlobalUnitSystem.get_units('pressure'))

standard_atmosphere


Changing the global unit system, e.g., making the nonsensical choice to set length to an energy unit, results in the validation to fail when defining positions with the units of angstrom. 

In [39]:
GlobalUnitSystem.set_global_units('length', unit.hartree)
pos = Positions(value=[[[1.0, 1.0, 1.0], [2.0, 2.0, 2.0], [3.0, 3.0, 3.0]]], units=unit.angstrom)

ValidationError: 1 validation error for Positions
  Value error, Unit angstrom of positions are not compatible with the property type length.
 [type=value_error, input_value={'value': [[[1.0, 1.0, 1....ts': <Unit('angstrom')>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

In [40]:
GlobalUnitSystem.set_global_units('length', unit.nanometer)

When hdf5 files are generated, quantities are automatically convert to the units specified in the `GlobalUnitSystem`. 


## Metadata

Meta data can be added to an individual record to provide additional details, such as smiles. 

In [41]:
smiles = MetaData(name='smiles', value='[CH]')

new_dataset.add_property("mol1", smiles)

If we print the record out, we now see that the smiles now appear under the `meta_data` subheading. 

In [42]:
mol1_rec = new_dataset.get_record('mol1')
print(mol1_rec)

name: mol1
* n_atoms: 2
* n_configs: 1
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[2, 2, 2],
        [3, 3, 3]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=1 n_atoms=2
* per-system properties (['energies', 'energies_disp']):
 -  name='energies' value=array([[0.1]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
 -  name='energies_disp' value=array([[0.03]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
* meta_data: (['smiles'])
 -  name='smiles' value='[CH]' units=<Unit('dimensionless')> classification='meta_data' property_type='meta_data' n_configs=None n_atoms=None



The `MetaData` pydantic model will allow users to specify a value that is either numpy array, list, string, float, or int.   Units can also be added to these values, however, validation and unit conversion will not occur on any meta data. 

In [43]:
run_temp = MetaData(name='temperature_of_simulation', value=300, units=unit.kelvin)

new_dataset.add_property('mol1', run_temp)

In [44]:
mol1_rec = new_dataset.get_record('mol1')
print(mol1_rec)

name: mol1
* n_atoms: 2
* n_configs: 1
* atomic_numbers:
 -  name='atomic_numbers' value=array([[1],
       [6]]) units=<Unit('dimensionless')> classification='atomic_numbers' property_type='atomic_numbers' n_configs=None n_atoms=2
* per-atom properties (['positions']):
 -  name='positions' value=array([[[2, 2, 2],
        [3, 3, 3]]]) units=<Unit('nanometer')> classification='per_atom' property_type='length' n_configs=1 n_atoms=2
* per-system properties (['energies', 'energies_disp']):
 -  name='energies' value=array([[0.1]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
 -  name='energies_disp' value=array([[0.03]]) units=<Unit('hartree')> classification='per_system' property_type='energy' n_configs=1 n_atoms=None
* meta_data: (['smiles', 'temperature_of_simulation'])
 -  name='smiles' value='[CH]' units=<Unit('dimensionless')> classification='meta_data' property_type='meta_data' n_configs=None n_atoms=None
 -  name='temperature_o